In [1]:
### This python code calculates mass isotopologue distribution (MID) from raw mass spectrometry data
# Written by Xin Guan (github.com/x-guan)

# basics
import numpy as np
import pandas as pd
import datetime

In [2]:
### determine monoisotopic m/z and charge from structure
# function - get_monoisotopic_mz_and_z

# for parsing ion structures
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import rdmolops
# input: acutumine.sdf
# return: monoiso_mz, charge

# input structure from a sdf file
structure = "acutumine.sdf"
mol = Chem.SDMolSupplier(structure)[0]
print(mol)

# determine properties of mol
monoiso_mz = rdMolDescriptors.CalcExactMolWt(mol)
print("m/z (mono isotope): ", monoiso_mz)
charge = rdmolops.GetFormalCharge(mol)
print("charge: ", charge)   # charge have to be +1

m/z (mono isotope):  398.13649162008994
charge:  1


In [3]:
### determine mz_windows from monoiso_mz
# function - calc_mz_windows

# input
C13_NEUTRON = 13.0033548378 - 12
H2_NEUTRON = 2.01410177811 - 1.00782503224
span = max_neutrons = 5
ppm=100
# return: mz_windows

# initialize output matrix
mz_windows = np.zeros(shape=(span+1, 2), dtype='float')
row_idxs = range(span+1)

# calc ppm factor
low_ppm = 1-ppm/1e6
high_ppm = 1+ppm/1e6

# loop through rows
for m in row_idxs:
    mz_m = np.array((monoiso_mz + m*C13_NEUTRON, monoiso_mz + m*H2_NEUTRON))   # double brakets
    mz_windows[m, :] = np.min(mz_m * low_ppm), np.max(mz_m * high_ppm)
print("m/z window: \n", mz_windows)

m/z window: 
 [[398.09667797 398.17630527]
 [399.09993247 399.18268264]
 [400.10318698 400.18906002]
 [401.10644148 401.19543739]
 [402.10969598 402.20181476]
 [403.11295048 403.20819214]]


In [4]:
### calculate MID raw data
# function - extract_mid_from_file

# read mzml files
from pyteomics import mzml
# input
rt_window = (5, 5.5)   # from 5 to 5.5 min
mzml_file = "mca_1_l_d2o.mzML"
# return: mean_mz, total_i

with mzml.read(mzml_file) as reader:
    # identify rt window
    rt_min, rt_max = tuple(rt_window)
    print("m/z window is from", rt_min, "to", rt_max, "min")
    
    # initialize output matrix of mean mz and intensities
    n_rows = mz_windows.shape[0]
    m_span = range(n_rows)
    
    # initialize lists to store relevant raw data points
    mzs = [[] for m in m_span]
    intensities = [[] for m in m_span]
    
    # loop through scans, keeping data points in mz_windows and rt_window only
    for spec in reader:
        rt = spec['scanList']['scan'][0]['scan start time']
        if rt >= rt_min and rt <= rt_max:
                # get raw scan data
                these_mzs = spec['m/z array']
                these_intensities = spec['intensity array']
                
                # index into mz_windows to find relevant data in scan
                index_mat = np.searchsorted(these_mzs, mz_windows)
                start = index_mat[:, 0]
                stop = index_mat[:, 1]
                
                for m in m_span:
                    # if scan has no mz values of interest, skip it
                    if start[m] != stop[m]:
                        mzs[m].extend(list(these_mzs[start[m]:stop[m]]))
                        intensities[m].extend(list(these_intensities[start[m]:stop[m]]))
    mean_mz = np.asarray([np.average(mzs[m], weights=intensities[m]) for m in m_span])
    print("mean m/z: ", mean_mz)
    total_i = np.asarray([np.sum(intensities[m]) for m in m_span])
    print("intensity: ", total_i)

m/z window is from 5 to 5.5 min
mean m/z:  [398.13487126 399.1388297  400.13299635 401.1366636  402.13899632
 403.14255773]
intensity:  [1.77147347e+08 4.14338471e+07 6.41503511e+07 1.45282674e+07
 2.35509175e+06 3.96237917e+05]


In [5]:
### preprare output
today = datetime.datetime.today()
m = np.arange(max_neutrons + 1)
C13_theo_mz = monoiso_mz + m*C13_NEUTRON
H2_theo_mz = monoiso_mz + m*H2_NEUTRON

out_df = pd.DataFrame({'m': m,
                       'mzml_file': mzml_file,
                       'structure': structure,
                       'analysis_date': today,
                       'C13_theo_mz': C13_theo_mz,
                       'H2_theo_mz': H2_theo_mz,
                       'mean_mz': mean_mz,
                       'raw_intensity': total_i,
                       'mid': total_i / total_i.sum()
                      })
print(out_df)
# out_df.to_csv("acutumine.csv")

   m         mzml_file      structure              analysis_date  C13_theo_mz  \
0  0  mca_1_l_d2o.mzML  acutumine.sdf 2019-01-09 13:50:18.813122   398.136492   
1  1  mca_1_l_d2o.mzML  acutumine.sdf 2019-01-09 13:50:18.813122   399.139846   
2  2  mca_1_l_d2o.mzML  acutumine.sdf 2019-01-09 13:50:18.813122   400.143201   
3  3  mca_1_l_d2o.mzML  acutumine.sdf 2019-01-09 13:50:18.813122   401.146556   
4  4  mca_1_l_d2o.mzML  acutumine.sdf 2019-01-09 13:50:18.813122   402.149911   
5  5  mca_1_l_d2o.mzML  acutumine.sdf 2019-01-09 13:50:18.813122   403.153266   

   H2_theo_mz     mean_mz  raw_intensity       mid  
0  398.136492  398.134871   1.771473e+08  0.590469  
1  399.142768  399.138830   4.143385e+07  0.138108  
2  400.149045  400.132996   6.415035e+07  0.213827  
3  401.155322  401.136664   1.452827e+07  0.048426  
4  402.161599  402.138996   2.355092e+06  0.007850  
5  403.167875  403.142558   3.962379e+05  0.001321  
